
1. **Data Cleaning and Validation** **bold text**
-Identify and Handle Missing or Inconsistent Data
Steps:

Load the dataset into a Pandas DataFrame.
Check for missing values (NaN or empty entries) using .isnull() or .info().
Decide how to handle missing values:
For critical fields like Entry_Price or SL:
Drop rows if missing values are significant and can't be imputed.
Impute values based on statistical measures (e.g., mean, median) if appropriate.
For non-critical fields: Replace missing values with placeholders like 'Unknown' or '0'.
Standardize inconsistent values, such as ensuring numeric columns don't have text entries.


-Check for Duplicate Records Steps:

Identify duplicate rows using .duplicated(). Drop duplicates if they are redundant for analysis. Use the subset parameter to check for duplication in specific columns (e.g., Trade_ID, Entry_Time). Validate remaining data to ensure no critical information is lost

In [ ]:
import pandas as pd

# Load dataset
file_path = '/content/trading_data.csv'  # Update with your file path
data = pd.read_csv(file_path)

# Step 1: Check for missing values
print("Missing values per column:")
print(data.isnull().sum())

# Step 2: Handle missing values
# Drop rows with missing values in 'Entry_Price' or 'SL'
if 'Entry_Price' in data.columns and 'SL' in data.columns:
    data = data.dropna(subset=['Entry_Price', 'SL'])

# Check if 'Notes' column exists before filling missing values
if 'Notes' in data.columns:
    data['Notes'] = data['Notes'].fillna('Unknown')

# Step 3: Convert columns to numeric where applicable
# Ensure 'Entry_Price' and 'SL' are numeric
if 'Entry_Price' in data.columns:
    data['Entry_Price'] = pd.to_numeric(data['Entry_Price'], errors='coerce')
if 'SL' in data.columns:
    data['SL'] = pd.to_numeric(data['SL'], errors='coerce')

# Drop rows with newly created NaNs due to invalid numeric conversion
data = data.dropna(subset=['Entry_Price', 'SL'])

# Step 4: Verify the changes
print("Cleaned data preview:")
print(data.head())

# Step 1: Check for duplicates
duplicates = data[data.duplicated()]
print(f"Number of duplicate rows: {len(duplicates)}")

# Step 2: Drop duplicate rows
data = data.drop_duplicates()

# Optionally, drop duplicates based on specific columns (e.g., Trade_ID and Entry_Time)
# data = data.drop_duplicates(subset=['Trade_ID', 'Entry_Time'])

# Verify duplicates removal
print(f"Data after removing duplicates: {data.shape}")


2 *Data Summarization**

-- Data Summarization
Step 1: Calculate Total Profit/Loss
The profit/loss can be calculated as the difference between Exit_Price and Entry_Price for all trades.
Formula:
Profit/Loss
=
(
Exit_Price
−
Entry_Price
)
×
Quantity
Profit/Loss=(Exit_Price−Entry_Price)×Quantity

--Create a Pivot Table
Summarize the average SL% and Trgt% grouped by the Index column.

In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/trading_data.csv'
df = pd.read_csv(file_path)

# Calculate profit/loss for each trade
df['Profit_Loss'] = (df['Exit_Price'] - df['Entry_Price']) * df['Quantity']

# Calculate total profit/loss
total_profit_loss = df['Profit_Loss'].sum()
print(f"Total Profit/Loss: {total_profit_loss}")

# Create pivot table
pivot_table = df.pivot_table(
    values=['SL%', 'Trgt%'],
    index='Index',
    aggfunc='mean'
)

print("Pivot Table - Average SL% and Trgt% by Index:")
print(pivot_table)




3.Sorting and Filtering

--Sort the data to show the top 10 trades by profit (or loss).Step 1: Sort Top 10 Trades by Profit/Loss
Sort the trades by Profit_Loss in descending order to get the top 10 most profitable trades.
Alternatively, sort in ascending order to get the top 10 biggest losses
--Filter the data to display only trades that hit their SL or Target. Filter Trades That Hit SL or Target
Filter rows where Hit_SL or Hit_Target column equals True.

In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv('/content/trading_data.csv')

# Calculate profit or loss for each trade, considering quantity
data['Profit/Loss'] = (data['Exit_Price'] - data['Entry_Price']) * data['Quantity']

# Sort the dataset by profit/loss and display the top 10 trades
sorted_data = data.sort_values(by='Profit/Loss', ascending=False).head(10)
print("Top 10 trades by profit:")
print(sorted_data[['Index', 'Entry_Price', 'Exit_Price', 'Quantity', 'Profit/Loss']])

# Ensure 'Exit_Reason' column is clean and standardized
data['Exit_Reason'] = data['Exit_Reason'].str.strip().str.lower()

# Filter trades that hit SL or Target based on 'Exit_Reason'
sl_target_hits = data[data['Exit_Reason'].isin(['trgt_hit', 'straddle_max_loss_hit', 'max_loss_hit'])]
print("\nTrades that hit SL or Target:")
print(sl_target_hits[['Index', 'Exit_Reason', 'Entry_Price', 'Exit_Price', 'Quantity', 'Profit/Loss']])


**4 Performance Analysis**

--Step 1: Calculate the Percentage of Trades That Hit SL vs. Target
Count the total number of trades.
Count how many trades hit SL and how many hit Target.
Calculate the percentage using:
Percentage
=
Count of SL or Target Trades
Total Trades
×
100
Percentage=
Total Trades
Count of SL or Target Trades
​

----Step 2: Group Data by Strategy and Determine Success Rate
Group the data by the Strategy column.
Define a "successful trade" as one that hits the target (i.e., Hit_Target == True).
Calculate the success rate for each strategy:
Success Rate
=
Number of Successful Trades
Total Trades for Strategy
×
100
Success Rate=
Total Trades for Strategy
Number of Successful Trades
​
 ×100

 ---Step 3: Analyze Correlation Between VixEntry, VixExit, and Trade Outcomes
Correlation indicates how strongly two variables are related (values range from -1 to 1).
Use pandas.corr() to calculate the correlation between VixEntry, VixExit, and the Profit_Loss.

In [ ]:
total_trades = len(data)

# Count trades hitting SL and Target
sl_hits = len(data[data['Exit_Reason'].isin(['straddle_max_loss_hit', 'max_loss_hit'])])
target_hits = len(data[data['Exit_Reason'] == 'trgt_hit'])

# Calculate percentages
sl_percentage = (sl_hits / total_trades) * 100
target_percentage = (target_hits / total_trades) * 100

print(f"\nPercentage of trades that hit SL: {sl_percentage:.2f}%")
print(f"Percentage of trades that hit Target: {target_percentage:.2f}%")

strategy_success = data.groupby('Strategy')['Profit/Loss'].mean()
print("\nSuccess rate by strategy (mean profit/loss):")
print(strategy_success)

# Correlation between VixEntry/VixExit and trade outcomes
correlation_matrix = data[['VixEntry', 'VixExit', 'Profit/Loss']].corr()
print("\nCorrelation between VixEntry, VixExit, and trade outcomes:")
print(correlation_matrix)


**5 Time-Based Analysis**

Analyze the performance of trades based on Date or Day.
--Step 1: Analyze Performance by Date or Day
Convert the Date column to a datetime object using pd.to_datetime().
Group data by Date or day of the week and calculate metrics like total profit/loss and trade count.

---Evaluate the average time a trade was open by calculating the difference between Entry_Time and Exit_Time.

Ensure Entry_Time and Exit_Time are in the correct datetime format.
Calculate the difference between Exit_Time and Entry_Time for each trade.
Convert the duration into hours or minutes and find the average holding tim

In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv('/content/trading_data.csv')

# Ensure date and time columns are properly parsed
data['Entry_Time'] = pd.to_datetime(data['Entry_Time'], errors='coerce', format='mixed')
data['Exit_Time'] = pd.to_datetime(data['Exit_Time'], errors='coerce', format='mixed')
data['Date'] = data['Entry_Time'].dt.date

# Drop rows with invalid datetime parsing
data = data.dropna(subset=['Entry_Time', 'Exit_Time'])

# Analyze performance based on Date
data['Profit/Loss'] = (data['Exit_Price'] - data['Entry_Price']) * data['Quantity']
performance_by_date = data.groupby('Date')['Profit/Loss'].mean()
print("\nPerformance by Date (mean profit/loss):")
print(performance_by_date)

# Analyze performance based on Day of the week
data['Day'] = data['Entry_Time'].dt.day_name()
performance_by_day = data.groupby('Day')['Profit/Loss'].mean()
print("\nPerformance by Day (mean profit/loss):")
print(performance_by_day)

# Calculate the average time a trade was open
data['Trade_Duration'] = (data['Exit_Time'] - data['Entry_Time']).dt.total_seconds() / 60  # Duration in minutes
average_trade_duration = data['Trade_Duration'].mean()

print(f"\nAverage trade duration: {average_trade_duration:.2f} minutes")


**6.Volatility Impact**

Compare Entry_imp_vol and Exit_imp_vol to understand the impact of implied volatility changes on trade outcomes.
Step 1: Analyze the Impact of Implied Volatility (Entry_imp_vol vs. Exit_imp_vol)
Compare Entry_imp_vol and Exit_imp_vol for profitable vs. losing trades:
Calculate the mean and standard deviation of Entry_imp_vol and Exit_imp_vol for both profitable and losing trades.
Analyze whether a significant change in implied volatility corresponds to profitable trades.

Analyze how Entry_delta and Exit_delta shifted in profitable vs. losing trades.
--Step 2: Analyze Entry_delta and Exit_delta for Profitable vs. Losing Trades
Calculate the mean and standard deviation of Entry_delta and Exit_delta for profitable and losing trades.
Compare the shifts in deltas between the two groups to identify patterns.

In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/trading_data.csv'
df = pd.read_csv(file_path)

# Calculate Profit/Loss for each trade
df['Profit_Loss'] = (df['Exit_Price'] - df['Entry_Price']) * df['Quantity']

# Separate profitable and losing trades
profitable_trades = df[df['Profit_Loss'] > 0]
losing_trades = df[df['Profit_Loss'] <= 0]

# Calculate stats for implied volatility
vol_stats = {
    "Profitable Trades": {
        "Avg_Entry_imp_vol": profitable_trades['Entry_imp_vol'].mean(),
        "Avg_Exit_imp_vol": profitable_trades['Exit_imp_vol'].mean(),
    },
    "Losing Trades": {
        "Avg_Entry_imp_vol": losing_trades['Entry_imp_vol'].mean(),
        "Avg_Exit_imp_vol": losing_trades['Exit_imp_vol'].mean(),
    }
}

print("Implied Volatility Analysis:")
print(vol_stats)
# Calculate stats for delta
delta_stats = {
    "Profitable Trades": {
        "Avg_Entry_delta": profitable_trades['Entry_delta'].mean(),
        "Avg_Exit_delta": profitable_trades['Exit_delta'].mean(),
    },
    "Losing Trades": {
        "Avg_Entry_delta": losing_trades['Entry_delta'].mean(),
        "Avg_Exit_delta": losing_trades['Exit_delta'].mean(),
    }
}

# Print the delta analysis
print("Delta Analysis:")
print(delta_stats)


7.Statistical Insights

Conduct a regression analysis to evaluate the relationship between IndexEntrySpot and IndexExitSpot.
--Step 1: Regression Analysis Between IndexEntrySpot and IndexExitSpot
Perform a simple linear regression to analyze the relationship between IndexEntrySpot and IndexExitSpot.
Use scipy.stats or statsmodels to calculate the regression coefficient, p-value, and R-squared to determine the strength of the relationship.


Use statistical formulas to determine if there's a significant difference in profit/loss between Expiry and Non-Expiry trades.
--Step 2: Compare Profit/Loss Between Expiry and Non-Expiry Trades
Categorize trades into "Expiry" and "Non-Expiry" based on the Expiry column.
Use a statistical test (e.g., a two-sample t-test) to determine if there’s a significant difference in Profit_Loss between these categories.

In [ ]:
from scipy.stats import linregress
import pandas as pd

# Ensure columns are numeric, replacing invalid entries with NaN
df['IndexEntrySpot'] = pd.to_numeric(df['IndexEntrySpot'], errors='coerce')
df['IndexExitSpot'] = pd.to_numeric(df['IndexExitSpot'], errors='coerce')

# Drop rows with NaN values in these columns
cleaned_df = df.dropna(subset=['IndexEntrySpot', 'IndexExitSpot'])

# Perform regression if enough data exists
if not cleaned_df.empty:
    slope, intercept, r_value, p_value, std_err = linregress(
        cleaned_df['IndexEntrySpot'], cleaned_df['IndexExitSpot']
    )
    print("Regression Analysis:")
    print(f"Slope: {slope:.2f}")
    print(f"Intercept: {intercept:.2f}")
    print(f"R-squared: {r_value**2:.2f}")
    print(f"P-value: {p_value:.2e}")
else:
    print("Insufficient valid data for regression analysis.")

from scipy.stats import ttest_ind

# Ensure 'Profit_Loss' column is calculated
if 'Profit_Loss' not in df.columns:
    df['Profit_Loss'] = (df['Exit_Price'] - df['Entry_Price']) * df['Quantity']

# Map 'Expiry/Non Expiry' to boolean values for filtering
df['Expiry'] = df['Expiry/Non Expiry'].map({'Expiry': True, 'Non Expiry': False})

# Separate trades into Expiry and Non-Expiry categories
expiry_trades = df[df['Expiry'] == True]['Profit_Loss']
non_expiry_trades = df[df['Expiry'] == False]['Profit_Loss']

# Perform the t-test
t_stat, p_value = ttest_ind(expiry_trades, non_expiry_trades, equal_var=False)

# Print the results
print("T-test for Expiry vs Non-Expiry Trades:")
print(f"T-statistic: {t_stat:.2f}")
print(f"P-value: {p_value:.2e}")

if p_value < 0.05:
    print("Significant difference in Profit/Loss between Expiry and Non-Expiry trades.")
else:
    print("No significant difference in Profit/Loss between Expiry and Non-Expiry trades.")



8.Risk and Reward

--Compute the risk/reward ratio for each trade using SL% and Trgt%.

Step 1: Compute Risk/Reward Ratio for Each Trade
Use SL% (Stop Loss percentage) and Trgt% (Target percentage) to compute the risk/reward ratio for each trade
Add a new column in the dataset to store the calculated ratio.

--Analyze how SL_move influenced trade outcomes.
Step 2: Analyze the Impact of SL_move on Trade Outcomes
Separate trades into two groups: those with SL_move adjustments and those without.
Compare the average Profit_Loss between the two groups.
Identify if SL_move improved or worsened trade outcom

In [ ]:
# Calculate Risk/Reward ratio for every trade
df['Risk_Reward_Ratio'] = df['SL%'] / df['Trgt%']

# Display the first few rows with the calculated Risk/Reward ratio
print("Risk/Reward Ratio for Each Trade:")
print(df[['SL%', 'Trgt%', 'Risk_Reward_Ratio']].head())

# Add a new column to indicate if SL_move influenced the outcome
df['SL_Move_Impact'] = df['SL_move'] > 0

# Analyze how SL_move influenced trade outcomes
sl_move_analysis = df.groupby('SL_Move_Impact')['Profit_Loss'].mean().to_dict()

# Print detailed trade analysis based on SL_move
print("\nImpact of SL Move on Trade Outcomes (for every trade):")
print("Average Profit/Loss by SL Move:")
for sl_move, avg_profit in sl_move_analysis.items():
    sl_move_status = "SL Moved" if sl_move else "No SL Movement"
    print(f"{sl_move_status}: {avg_profit:.2f}")

# Display the first few rows to verify added columns
print("\nUpdated Data with SL_Move_Impact and Risk/Reward Ratio:")
print(df[['SL%', 'Trgt%', 'Risk_Reward_Ratio', 'SL_Move_Impact', 'Profit_Loss']].head())


9.Instrument Comparison

--Compare the performance of InstrumentLeg for Call (CE) vs. Put (PE) trades.
Step 1: Compare Performance of InstrumentLeg for Call (CE) vs. Put (PE)
Filter the dataset into two subsets:
Trades with InstrumentLeg as Call (CE).
Trades with InstrumentLeg as Put (PE).
Calculate the average Profit_Loss for both subsets.
Analyze which instrument performed better overall.


--Determine if ce_strike trades performed better than pe_strike trades.
Step 2: Compare Performance of ce_strike vs. pe_strike Trades
Identify trades involving ce_strike and pe_strike.
Calculate the total and average profit/loss for these trades.
Determine if ce_strike or pe_strike performed better


In [ ]:
# Filter trades by InstrumentLeg
call_trades = df[df['InstrumentLeg'] == 'CE']
put_trades = df[df['InstrumentLeg'] == 'PE']

# Calculate average profit/loss for each instrument
instrument_comparison = {
    "Call Trades (CE)": call_trades['Profit_Loss'].mean(),
    "Put Trades (PE)": put_trades['Profit_Loss'].mean(),
}

print("Instrument Comparison (CE vs. PE):")
print(instrument_comparison)

# Check for missing values in ce_strike and pe_strike before filtering
if 'ce_strike' in df.columns and 'pe_strike' in df.columns:
    ce_strike_trades = df[df['ce_strike'].notna()]
    pe_strike_trades = df[df['pe_strike'].notna()]

    # Calculate performance metrics for CE and PE strike trades
    strike_comparison = {
        "CE Strike Trades": {
            "Total Profit/Loss": ce_strike_trades['Profit_Loss'].sum(),
            "Average Profit/Loss": ce_strike_trades['Profit_Loss'].mean(),
        },
        "PE Strike Trades": {
            "Total Profit/Loss": pe_strike_trades['Profit_Loss'].sum(),
            "Average Profit/Loss": pe_strike_trades['Profit_Loss'].mean(),
        },
    }

    print("Strike Comparison (CE vs. PE):")
    print(strike_comparison)
else:
    print("Columns 'ce_strike' or 'pe_strike' not found in the dataset.")


10.Charts and Dashboards

Create a chart showing the performance of different Strategies over time.
Develop a dashboard to monitor key metrics like success rate, average profit/loss, and strategy performance.

Chart:
Here’s how the dashboard works:

Data Loading & Processing:

The dataset is read into a DataFrame.
Profit/Loss is calculated for each trade based on entry/exit prices, quantity, and direction (Buy/Sell).
Key Metrics:

Metrics like success rate, average profit/loss, total profit/loss, and total trades are computed for each strategy.
Interactive Dashboard:

Filters: Dropdown for selecting a strategy and a date range picker.
Dynamic Charts:
Line chart shows strategy performance over time.
Bar chart displays daily profit/loss.
Callbacks:

User interactions (selecting strategy or date range) trigger a callback to filter data and update metrics/charts.
Execution:

The dashboard is hosted using jupyter-dash and runs interactively within the Colab notebook.








In [ ]:
!pip install pandas plotly dash jupyter-dash


In [ ]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output

# Load and preprocess the dataset
file_path = '/content/trading_data.csv'  # Update to your uploaded file's path
df = pd.read_csv(file_path)

df['Date'] = pd.to_datetime(df['Date'])
df['Direction'] = df['Direction'].map({'Buy': 1, 'Sell': -1})
df['Profit_Loss'] = (df['Exit_Price'] - df['Entry_Price']) * df['Quantity'] * df['Direction']

# Key metrics calculations
key_metrics = df.groupby('Strategy').agg(
    Success_Rate=('Profit_Loss', lambda x: (x > 0).mean() * 100),
    Avg_Profit_Loss=('Profit_Loss', 'mean'),
    Total_Profit_Loss=('Profit_Loss', 'sum'),
    Total_Trades=('Profit_Loss', 'count')
).reset_index()

strategy_performance = df.groupby(['Strategy', 'Date'])['Profit_Loss'].sum().reset_index()

# Initialize JupyterDash app
app = JupyterDash(__name__)

# App layout
app.layout = html.Div([
    html.H1("Trading Strategy Dashboard", style={'text-align': 'center'}),

    # Dropdown for strategy selection
    html.Div([
        html.Label("Select Strategy:"),
        dcc.Dropdown(
            id='strategy-dropdown',
            options=[{'label': s, 'value': s} for s in df['Strategy'].unique()],
            value=df['Strategy'].unique()[0],
            multi=False
        ),
    ], style={'margin-bottom': '20px'}),

    # Date range picker
    html.Div([
        html.Label("Select Date Range:"),
        dcc.DatePickerRange(
            id='date-picker',
            start_date=df['Date'].min(),
            end_date=df['Date'].max(),
            display_format='YYYY-MM-DD'
        )
    ], style={'margin-bottom': '20px'}),

    # Output key metrics
    html.Div(id='key-metrics-output', style={'margin-bottom': '20px'}),

    # Charts
    dcc.Graph(id='performance-chart'),
    dcc.Graph(id='metrics-chart')
])

# Callbacks for interactivity
@app.callback(
    [Output('key-metrics-output', 'children'),
     Output('performance-chart', 'figure'),
     Output('metrics-chart', 'figure')],
    [Input('strategy-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def update_dashboard(selected_strategy, start_date, end_date):
    # Filter data
    filtered_data = df[
        (df['Strategy'] == selected_strategy) &
        (df['Date'] >= pd.to_datetime(start_date)) &
        (df['Date'] <= pd.to_datetime(end_date))
    ]

    # Update key metrics
    success_rate = (filtered_data['Profit_Loss'] > 0).mean() * 100
    avg_profit_loss = filtered_data['Profit_Loss'].mean()
    total_profit_loss = filtered_data['Profit_Loss'].sum()
    total_trades = filtered_data['Profit_Loss'].count()

    key_metrics_output = html.Div([
        html.H4(f"Key Metrics for {selected_strategy}:"),
        html.P(f"Success Rate: {success_rate:.2f}%"),
        html.P(f"Average Profit/Loss: {avg_profit_loss:.2f}"),
        html.P(f"Total Profit/Loss: {total_profit_loss:.2f}"),
        html.P(f"Total Trades: {total_trades}")
    ])

    # Update performance chart
    performance_data = strategy_performance[
        (strategy_performance['Strategy'] == selected_strategy) &
        (strategy_performance['Date'] >= pd.to_datetime(start_date)) &
        (strategy_performance['Date'] <= pd.to_datetime(end_date))
    ]
    performance_fig = px.line(
        performance_data,
        x='Date',
        y='Profit_Loss',
        title=f"Performance Over Time ({selected_strategy})",
        labels={'Profit_Loss': 'Profit/Loss', 'Date': 'Date'}
    )

    # Update metrics chart
    metrics_data = filtered_data.groupby(['Strategy', 'Date'])['Profit_Loss'].sum().reset_index()
    metrics_fig = px.bar(
        metrics_data,
        x='Date',
        y='Profit_Loss',
        color='Strategy',
        title="Daily Profit/Loss",
        labels={'Profit_Loss': 'Profit/Loss', 'Date': 'Date'}
    )

    return key_metrics_output, performance_fig, metrics_fig

# Run the app in Colab
app.run_server(mode='inline')


11.Error Detection

Identify any anomalies or outliers in the data (e.g., trades with extremely high or low profits).
Discuss how you would validate if Entry_PCR and Exit_PCR values align with market conditions.

12.Optimization Suggestions

Based on your analysis, suggest ways to optimize SL, Target, or entry/exit conditions to improve the strategy’s performance.


--Error Detection & Outlier Analysis:

Identifies outliers in Profit_Loss using the 3-sigma rule.
Detects anomalies in Entry_PCR and Exit_PCR values based on predefined thresholds.
SL% and Target% Analysis:

Compares Stop Loss (SL%) and Target (Trgt%) between winning and losing trades.
Computes correlations between Entry_delta, Exit_delta, and Profit_Loss.
Visualizations:

Boxplots and scatterplots to highlight outliers, PCR trends, and deltas' impact.
Optimization Suggestions:

Summarizes actionable strategies to improve performance based on the analysis.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
file_path = '/content/trading_data.csv'  # Adjust this path if needed
df = pd.read_csv(file_path)

# Preprocess data
df['Date'] = pd.to_datetime(df['Date'])
df['Direction'] = df['Direction'].map({'Buy': 1, 'Sell': -1})
df['Profit_Loss'] = (df['Exit_Price'] - df['Entry_Price']) * df['Quantity'] * df['Direction']

# Define function for error detection and analysis
def error_detection_analysis(data):
    print("=== Error Detection ===")
    # Calculate mean and standard deviation
    mean_profit_loss = data['Profit_Loss'].mean()
    std_profit_loss = data['Profit_Loss'].std()

    # Define outlier thresholds
    lower_bound = mean_profit_loss - 3 * std_profit_loss
    upper_bound = mean_profit_loss + 3 * std_profit_loss

    # Identify outliers
    outliers = data[(data['Profit_Loss'] < lower_bound) | (data['Profit_Loss'] > upper_bound)]
    print(f"Outliers detected: {len(outliers)}")
    print(outliers)

    # Visualize with a boxplot
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=data['Profit_Loss'])
    plt.title('Boxplot of Profit/Loss to Identify Outliers')
    plt.show()

    # Check for anomalies in PCR values
    anomalous_pcr = data[(data['Entry_PCR'] < 0) | (data['Exit_PCR'] < 0) |
                         (data['Entry_PCR'] > 10) | (data['Exit_PCR'] > 10)]
    print(f"Anomalies in PCR values: {len(anomalous_pcr)}")
    print(anomalous_pcr)

    # PCR trends visualization
    sns.scatterplot(data=data, x='Entry_PCR', y='Profit_Loss', hue='Strategy')
    plt.title('Entry_PCR vs. Profit/Loss')
    plt.show()

    sns.scatterplot(data=data, x='Exit_PCR', y='Profit_Loss', hue='Strategy')
    plt.title('Exit_PCR vs. Profit/Loss')
    plt.show()

# Analyze SL%, Target%, and deltas
def analyze_sl_target(data):
    print("=== SL and Target Analysis ===")
    # Winning vs Losing trades
    winning_trades = data[data['Profit_Loss'] > 0]
    losing_trades = data[data['Profit_Loss'] <= 0]

    avg_sl_winning = winning_trades['SL%'].mean()
    avg_trgt_winning = winning_trades['Trgt%'].mean()
    avg_sl_losing = losing_trades['SL%'].mean()
    avg_trgt_losing = losing_trades['Trgt%'].mean()

    print(f"Average SL% in Winning Trades: {avg_sl_winning:.2f}")
    print(f"Average Target% in Winning Trades: {avg_trgt_winning:.2f}")
    print(f"Average SL% in Losing Trades: {avg_sl_losing:.2f}")
    print(f"Average Target% in Losing Trades: {avg_trgt_losing:.2f}")

    # Correlation analysis
    corr_entry = data['Entry_delta'].corr(data['Profit_Loss'])
    corr_exit = data['Exit_delta'].corr(data['Profit_Loss'])

    print(f"Correlation between Entry_delta and Profit_Loss: {corr_entry:.2f}")
    print(f"Correlation between Exit_delta and Profit_Loss: {corr_exit:.2f}")

    # Visualize delta impact on profits
    sns.scatterplot(data=data, x='Entry_delta', y='Profit_Loss')
    plt.title('Entry_delta vs Profit/Loss')
    plt.show()

    sns.scatterplot(data=data, x='Exit_delta', y='Profit_Loss')
    plt.title('Exit_delta vs Profit/Loss')
    plt.show()

# Optimization suggestions
def suggest_optimizations(data):
    print("=== Optimization Suggestions ===")
    print("- Adjust SL% to average levels for winning trades.")
    print("- Focus on trades where PCR values align with historically profitable ranges.")
    print("- Analyze and optimize trades based on Entry/Exit deltas with high positive correlations.")
    print("- Prioritize instruments or strategies with consistently higher success rates.")

# Run the functions
error_detection_analysis(df)
analyze_sl_target(df)
suggest_optimizations(df)
